In [ ]:
pip install ultralytics onnxruntime opencv-python pyyaml numpy

In [ ]:
import sys
import subprocess

print("⬇️ Installing BoT-SORT and dependencies...")
# 1. Install helper libraries first to prevent errors
deps = ["filterpy", "lapx", "yacs", "ftfy", "regex", "loguru", "thop", "termcolor", "scikit-image", "gdown", "numpy<2.0"]
subprocess.run([sys.executable, "-m", "pip", "install"] + deps, stdout=subprocess.DEVNULL)

# 2. Install BoxMOT (Pinned version for stability)
subprocess.run([sys.executable, "-m", "pip", "install", "boxmot==10.0.43", "--no-deps"], stdout=subprocess.DEVNULL)

print("✅ Installation Complete. Now run the main code.")

In [ ]:
import sys
import subprocess

# 1. Uninstall the current (broken) numpy
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "numpy"], stdout=subprocess.DEVNULL)

# 2. Install a compatible version (1.26.4 is the safest bet)
subprocess.run([sys.executable, "-m", "pip", "install", "numpy==1.26.4"], stdout=subprocess.DEVNULL)

print("✅ NumPy fixed. Please RESTART your runtime/kernel now.")

In [ ]:
import cv2
import numpy as np
import onnxruntime as ort
from ultralytics import YOLO
from pathlib import Path
import torch
import os
import sys

# ==========================================
# 1. UNIVERSAL IMPORT (BoT-SORT)
# ==========================================
print("🔍 Initializing Ultimate BoT-SORT...")
BoTSORT_Class = None

def find_tracker():
    import importlib
    paths = ["boxmot", "boxmot.trackers.botsort.botsort", "boxmot.trackers.botsort.bot_sort"]
    names = ["BoTSORT", "BotSort", "BOTSORT"]
    for path in paths:
        try:
            module = importlib.import_module(path)
            for name in names:
                if hasattr(module, name): return getattr(module, name)
        except ImportError: continue
    return None

BoTSORT_Class = find_tracker()
if BoTSORT_Class is None:
    print("❌ Error: BoT-SORT not found.")
    sys.exit()

# ==========================================
# 2. BALANCED SETTINGS
# ==========================================
CONF_THRESH = 0.30
ASPECT_RATIO_MAX = 0.9    # Vehicle Killer
MIN_AREA = 2500           # Removes noise

# 🚨 BALANCED LOGIC (Same as Deep OC-SORT version)
NECK_ZONE_SIZE = 0.25     # Medium Zone 
MIN_SNATCH_FRAMES = 3     # 0.12s contact
MAX_SNATCH_FRAMES = 90    # 3.5s limit

# ==========================================
# 3. INITIALIZE TRACKER
# ==========================================
reid_weights = Path('osnet_x1_0_msmt17.pt') # 1. Using the BIG Model

tracker = BoTSORT_Class(
    model_weights=reid_weights,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    fp16=False,
    
    # 2. BoT-SORT Specific Stability Settings
    track_high_thresh=0.5,    # Only track clear people
    new_track_thresh=0.6,     # Hard to start new tracks
    match_thresh=0.7,         # Strict matching
    track_buffer=100,         # Remember lost IDs for 4s
    
    # 3. The "Secret Sauce" of BoT-SORT
    cmc_method="sparseOptFlow", # Camera Motion Compensation (GMC)
    appearance_thresh=0.25,     # Trust Re-ID more
    proximity_thresh=0.55       
)

# Load Pose & YOLO
RTMPOSE_PATH = '/kaggle/input/rtm-pose/onnx/default/1/rtmpose-m_ap10k_256.onnx' 
ort_sess = ort.InferenceSession(RTMPOSE_PATH, providers=['CPUExecutionProvider'])
in_name = ort_sess.get_inputs()[0].name

try:
    yolo = YOLO('/kaggle/input/best-yolov8m/pytorch/default/1/best _yolov8m.pt')
except:
    yolo = YOLO('yolo11s.pt')

# ==========================================
# 4. HELPER FUNCTIONS
# ==========================================
def preprocess_pose(img):
    resized = cv2.resize(img, (256, 256))
    mean, std = np.array([123.675, 116.28, 103.53]), np.array([58.395, 57.12, 57.375])
    return np.expand_dims(((cv2.cvtColor(resized, cv2.COLOR_BGR2RGB) - mean) / std).astype(np.float32).transpose(2, 0, 1), axis=0)

def get_kpts(outputs, h, w):
    simcc_x, simcc_y = outputs[0], outputs[1]
    x = np.argmax(simcc_x, axis=2)[0] / simcc_x.shape[2] * w
    y = np.argmax(simcc_y, axis=2)[0] / simcc_y.shape[2] * h
    return np.stack([x, y], axis=-1)

# ==========================================
# 5. MAIN PIPELINE
# ==========================================
VIDEO_PATH = '/kaggle/input/snatching-videos/snatching_videos/snatching_11.mp4'
OUTPUT_PATH = '/kaggle/working/final_botsort_ultimate.mp4'

if not os.path.exists(VIDEO_PATH):
    print("❌ Video missing.")
    sys.exit()

cap = cv2.VideoCapture(VIDEO_PATH)
W, H = int(cap.get(3)), int(cap.get(4))
FPS = cap.get(cv2.CAP_PROP_FPS) or 25
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), int(FPS), (W, H))

interaction_timers = {}
persistence_buffer = {} # Stores "Grace Frames"

print("🚀 Running ULTIMATE BoT-SORT Mode...")

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    frame_count += 1
    if frame_count % 20 == 0: print(f"Processing frame {frame_count}...", end='\r')
    
    # 1. DETECT
    results = yolo.predict(frame, conf=CONF_THRESH, verbose=False, classes=[0])
    dets_to_track = []
    
    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()
        clss = r.boxes.cls.cpu().numpy()
        for box, conf, cls in zip(boxes, confs, clss):
            x1, y1, x2, y2 = map(int, box[:4])
            w, h = (x2-x1), (y2-y1)
            if (w*h) < MIN_AREA: continue
            if (w / h) > ASPECT_RATIO_MAX: continue
            dets_to_track.append([x1, y1, x2, y2, conf, cls])
    
    dets_to_track = np.array(dets_to_track)
    
    # 2. TRACK (BoT-SORT)
    if len(dets_to_track) > 0:
        tracks = tracker.update(dets_to_track, frame)
    else:
        tracks = np.empty((0, 8))
        
    # 3. COLLECT DATA
    people = []
    for track in tracks:
        x1, y1, x2, y2 = map(int, track[:4])
        tid = int(track[4])
        
        # ZONE LOGIC
        neck_cx = int(x1 + (x2-x1)//2)
        neck_cy = int(y1 + (y2-y1)*0.15) 
        neck_rad = int((y2-y1)*NECK_ZONE_SIZE)
        
        # CHEST LINE (Vertical Filter)
        chest_y = int(y1 + (y2-y1)*0.40) 
        
        hands = []
        crop = frame[max(0,y1):min(H,y2), max(0,x1):min(W,x2)]
        if crop.size > 0:
            pose_in = preprocess_pose(crop)
            kpts = get_kpts(ort_sess.run(None, {in_name: pose_in}), crop.shape[0], crop.shape[1])
            for k in [9, 10]:
                if k < len(kpts):
                    hands.append((int(kpts[k][0]+x1), int(kpts[k][1]+y1)))
        
        people.append({'id': tid, 'box': (x1,y1,x2,y2), 'neck': (neck_cx,neck_cy,neck_rad), 'hands': hands, 'chest': chest_y})

    # 4. CROSS-CHECK
    snatch_alerts = set()
    
    for p1 in people: # Thief
        for p2 in people: # Victim
            if p1['id'] == p2['id']: continue
            pair_key = tuple(sorted((p1['id'], p2['id'])))
            
            vx, vy, vrad = p2['neck']
            victim_chest = p2['chest']
            
            is_touching = False
            for hx, hy in p1['hands']:
                dist = np.sqrt((hx-vx)**2 + (hy-vy)**2)
                
                # RULE: Must be CLOSE + ABOVE CHEST
                if dist < vrad and hy < victim_chest:
                    is_touching = True
                    cv2.line(frame, (hx, hy), (vx, vy), (0, 0, 255), 2)
                    break
            
            if is_touching:
                persistence_buffer[pair_key] = 5 # Reset Buffer
                
                if pair_key not in interaction_timers: interaction_timers[pair_key] = 0
                interaction_timers[pair_key] += 1
                
                if interaction_timers[pair_key] >= MIN_SNATCH_FRAMES:
                    snatch_alerts.add(p1['id'])
                    snatch_alerts.add(p2['id'])
            else:
                # 🛡️ PERSISTENCE LOGIC
                if pair_key in persistence_buffer and persistence_buffer[pair_key] > 0:
                    persistence_buffer[pair_key] -= 1
                    if interaction_timers.get(pair_key, 0) >= MIN_SNATCH_FRAMES:
                        snatch_alerts.add(p1['id'])
                        snatch_alerts.add(p2['id'])
                else:
                    if pair_key in interaction_timers:
                        interaction_timers[pair_key] = max(0, interaction_timers[pair_key] - 1)

    # 5. DRAW
    for p in people:
        tid = p['id']
        x1, y1, x2, y2 = p['box']
        color = (0, 255, 0)
        
        if tid in snatch_alerts:
            color = (0, 0, 255)
            cv2.putText(frame, "SNATCH!", (x1, y1-30), 0, 1.0, color, 3)
            nx, ny, nr = p['neck']
            cv2.circle(frame, (nx, ny), nr, color, 3)
            
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"ID:{tid}", (x1, y1-5), 0, 0.6, (255,255,0), 2)

    out.write(frame)

cap.release()
out.release()
print(f"✅ DONE: {OUTPUT_PATH}")